In [34]:
#IMPORTING REQUIRED LIBRARIES

import lxml
import re
import numpy as np
import pandas as pd
import time
import io
from google.colab import files
from datetime import date

from bs4 import BeautifulSoup
from requests import get

# **# Part 1 of the Task**

URLS for scrapping - use any two at a time

In [19]:
url1 = "https://www.justwatch.com/in/tv-shows"
url2 = "https://www.justwatch.com/in/movies"
#url1 = "https://www.justwatch.com/in/provider/netflix?sort_by=trending_7_day"
#url1 = "https://www.justwatch.com/in/provider/amazon-prime-video?sort_by=trending_7_day"
#url1 = "https://www.justwatch.com/in/provider/hotstar?sort_by=trending_7_day"

Code to fetch Movie/ TV Show details

In [17]:
class IMDB2(object):
	"""docstring for IMDB"""

	#Loading the page in beautiful soup library
	def __init__(self, url):
		super(IMDB2, self).__init__()
		page = get(url)
		self.soup = BeautifulSoup(page.content, 'lxml')

	#Loading the Page's Division/ Block from where datas would be scrapped
	def bodyContent(self):
		content = self.soup.find(id="app")
		return content.find_all("div", class_="jw-info-box__container")

	#Main code to fetch Movie Data
	def movieData(self, stri):

		#Define Variables
		movieFrame = self.bodyContent()
		movieTitle = []
		movieRunTime = []
		movieGenre = []
		movieRating = []
		movieDescription = []
		movieOTT = []
		list1 = []

		#Loop to fetch movie/ tv show details
		for movie in movieFrame:
			movieFirstLine = movie.find("div", class_="title-block__container").text
			movieTitle=(movieFirstLine)                                                 #Fetch Movie/ TV Show Title - Fetched format is "Name (Year) OriginalTitle"
			l=0
			for moviettt in movie.find_all("p", class_="mt-0"):
				movieOTT = moviettt.text

			#print(movieOTT)

			for mytag in movie.find_all("div", class_="detail-infos"):
				l = l+1
				if l == 1:
					movieRating=(mytag.text)                                                # Fetch rating - Fetched format is "JustWatch rating (in %) and IMDB rating (in x.y format) based out of (z) votes"
				if l == 2:
					movieGenre=(mytag.text)                                                # Fetch Genre - Fetched format is "Genres: Horror, Drama, Comedy.....etc"
				if l == 3:
					movieRunTime=(mytag.text)                                               # Fetch Run Time - Fetched format is "x h and yz min"
				if l == 4:
					movieDescription=(mytag.text)
			movieData1 = {"movieTitle" : [movieTitle] ,"movieRunTime" : [movieRunTime], "movieGenre" : [movieGenre], "movieRating" : [movieRating], "movieDescription": [movieDescription], "movieOTT": [movieOTT], "movieURL": stri}
			list1.append(movieData1)
			movieData = [movieTitle,movieRunTime, movieGenre, movieRating, movieDescription, movieOTT, stri]
		return movieData1

Code to fetch urls of each Movie or Tv Show

In [16]:
class IMDB(object):
	"""docstring for IMDB"""

	#Loading the page in beautiful soup library
	def __init__(self, url):
		super(IMDB, self).__init__()
		page = get(url)
		self.soup = BeautifulSoup(page.content, 'lxml')

	#Loading the Page's Division/ Block from where datas would be scrapped
	def bodyContent(self):
		content = self.soup.find(id="app")
		#print(content.find_all("div", class_="title-list-grid__item"))
		return content.find_all("div", class_="title-list-grid__item")

	#Main code to fetch movie/ tv show urls
	def movieData(self, inte):
		movieFrame = self.bodyContent()
		movieURL = []
		finalData = []
		fileno = inte

		for movie in movieFrame:
			movieURLSNAP = movie.find("a", class_="title-list-grid__item--link")
			#print(movieFirstLine.get("href"))
			movieURL = "https://www.justwatch.com" + movieURLSNAP.get("href")
			#print(movieTitle)
			site2 = IMDB2(movieURL)
			finalData.append(site2.movieData(movieURL))


		df = pd.DataFrame(finalData)   # Converting the fetched datas into DatFrame
		df.to_csv('Final.csv',index=False)
		d1 = pd.read_csv('Final.csv')

		#Variables
		var0 = []
		var1 = []
		var2 = []
		var3 = []
		var4 = []
		var5 = []
		var6 = []
		var7 = []
		var8 = []
		var9 = []
		var10 = []
		var11 = []


		#Format the fetched data as per requirements

		var0 = d1['movieTitle'].str.split(" ", n=1, expand=True)
		var1 = var0[1].str.split(" \(", n=1, expand = True)
		var2 = var1[1].str.split("\)", n=1, expand = True)
		var3 = d1['movieRating'].str.split("% ", n=1 , expand=True)
		var4 = var3[1].str.split(" \(", n=1, expand=True)
		var5 = var4[0].str.split("\'", n=1, expand=True)
		var6 = d1['movieRunTime'].str.split("Runtime", n=1 , expand=True)
		var7 = var6[1].str.split("\']", n=1 , expand=True)
		var8 = d1['movieGenre'].str.split("Genres", n=1 , expand=True)
		var9 = var8[1].str.split("\']", n=1 , expand=True)
		var10 = d1['movieOTT'].str.split("streaming on ", n=1, expand = True)
		var11 = var10[1].str.split(".", n=1, expand=True)

		d1['year'] = var2[0]
		d1['Title'] = var1[0]
		d1['Rating'] = var5[0]
		d1['movieRunTime'] = var7[0]
		d1['movieGenre'] = var9[0]
		d1['movieOTT'] = var11[0]
		d1 = d1.drop(["movieRating"], axis =1)
		d1 = d1.drop(["movieDescription"], axis =1)
		d1 = d1.drop(["movieTitle"], axis =1)

		#Download the CSV file
		filename = "List" + str(fileno) + ".csv"
		d1.to_csv(filename,index=False)
		files.download(filename)
		#print(d1)

		return finalData

Main

In [20]:
if __name__ == '__main__':
	site1 = IMDB(url1)						#Downloaded file is Final1.csv and this conatains details of top 40 TV Shows
	data = site1.movieData(1)

	site2 = IMDB(url2)						#Downloaded file is Final.csv  and this conatains details of top 40 Movies
	data1 = site2.movieData(2)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **This concludes part 1 of the task and now we head to part 2**

In [23]:
d11 = pd.read_csv('List1.csv')  #Reading webscrapped data into dataframe
d12 = pd.read_csv('List2.csv')  #Reading webscrapped data into dataframe

First filter out the data based on Movies/ TV shows which were released within last two years

In [29]:
d21 = d11[d11['year'].astype(int) >= date.today().year-2]  #Current year - 2
d22 = d12[d12['year'].astype(int) >= date.today().year-2]  #Current year - 2

Next Filter based on Movies/ TV Shows with rating above 7

In [30]:
d31 = d21[d21['Rating'].astype(float) > 7]     # Converting rating typr to float and filter tv shows with ratings above 7
d32 = d22[d22['Rating'].astype(float) > 7]     # Converting rating typr to float and filter movies with ratings above 7

Calculate the average rating of the filtered data

In [32]:
avg_rating1 = d31['Rating'].astype(float).mean()  #Find avaerage/ mean rating of TV Shows
avg_rating2 = d32['Rating'].astype(float).mean()  #Find avaerage/ mean rating of Movies

Top 5 Genres

In [27]:
modeGenre1 = d31['movieGenre'].value_counts().nlargest(5)  #Find top 5 genres of TV Shows
print(modeGenre1)
modeGenre2 = d32['movieGenre'].value_counts().nlargest(5)  #Find top 5 genres of Movies
print(modeGenre2)


Drama, Mystery & Thriller, Crime                        2
Drama, Romance                                          2
Drama, Crime, Mystery & Thriller, Action & Adventure    1
Drama, Crime, Mystery & Thriller                        1
Drama, Comedy                                           1
Name: movieGenre, dtype: int64
Mystery & Thriller, Action & Adventure                             3
Drama, History                                                     2
Comedy, Science-Fiction, Mystery & Thriller, Action & Adventure    1
Drama, Comedy, History                                             1
Mystery & Thriller, Action & Adventure, Crime, Horror              1
Name: movieGenre, dtype: int64


Top 3 Streaming service platform

In [38]:
streamPlatform1 = d31['movieOTT'].value_counts().nlargest(3)  #Find top 3 Streaming platforms for TV Shows
print(streamPlatform1)
streamPlatform2 = d32['movieOTT'].value_counts().nlargest(3)  #Find top 3 Streaming platforms for Movies
print(streamPlatform2)

### To find top most streaming service just replace nlargest(3) with nlargest(1)

Amazon Prime Video    3
Netflix               2
Apple TV Plus         2
Name: movieOTT, dtype: int64
Amazon Prime Video    3
Hotstar               2
Lionsgate Play        1
Name: movieOTT, dtype: int64


In [28]:
d31.to_csv("Final_List1.csv",index=False)
files.download("Final_List1.csv")
d32.to_csv("Final_List2.csv",index=False)
files.download("Final_List2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>